In [1]:
from openai import OpenAI

In [13]:
client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

system_prompt = '''You are a helpful assistant. Your job is to filter the user's text (which is a transcript of a conversation) and remove all filler, unnecessary and unrelated text. You must output text directly, such that it can be fed to another llm model. Fix incorrect transcripts to preserve technical knowledge. Just output '''

transcription = '''so I was thinking I'll start building a tool which is like a rubber duck, but which resides on your PC where you can converse with it to help you idea a debug code I'm not sure how to start building it so I am going to start maybe with a simple open eye template'''

user_prompt = f'''Here's the transcription, clean it up and preserve technical knowledge and return first person text: "{transcription}"'''

response1 = client.chat.completions.create(
    model="llama-3.2-3b-instruct",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
)

cleaned_idea = response1.choices[0].message.content
print(cleaned_idea)

I want to create a tool that functions like a virtual rubber duck, but in the form of a desktop application where users can interact with it to generate debug code ideas. I'll begin by designing a simple, open-eye template as the foundation of my project.


In [16]:
client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

system_prompt2 = '''You are a helpful assistant whose job is to guide the user in ideating a project or code approach. do not supply the answer but ask questions to guide the user. '''

user_prompt2 = f'''Here's the user's idea, filtered: "{cleaned_idea}
If you ask followup questions, make the questions normal and conversational. Do not ask all the questions at once.'''

response2 = client.chat.completions.create(
    model="llama-3.2-3b-instruct",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
)
duck_response = response2.choices[0].message.content

print(duck_response)

That sounds like a unique and fascinating project. I'm excited to help you brainstorm.

So, let's start with the basics. What kind of interactions do you envision users having with this virtual rubber duck? Will it be more like a conversation, where the user talks to it and it responds with code ideas, or will it be more like a game, where the user plays with it and gets suggestions?


In [2]:
# client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

# system_prompt = '''You are a helpful assistant. Your job is to filter the user's text (which is a transcript of a conversation) and remove all filler, unnecessary and unrelated text. You must output text directly, such that it can be fed to another llm model. Fix incorrect transcripts to preserve technical knowledge. Just output '''

# transcription = '''I want to build a restaurant management system in Java'''

# user_prompt = f'''Here's the transcription, clean it up and preserve technical knowledge and return first person text: "{transcription}"'''

# response1 = client.chat.completions.create(
#     model="llama-3.2-3b-instruct",
#     messages=[
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": user_prompt},
#     ],
# )

# cleaned_idea = response1.choices[0].message.content

# # Initialize conversation history
# conversation_history = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": user_prompt}
# ]

# system_prompt2 = '''You are a helpful assistant whose job is to guide the user in ideating a project or code approach. do not supply the answer but ask questions to guide the user. Keep the conversation going until reaching a concrete implementation plan.'''

# user_prompt2 = f'''Here's the user's idea, filtered: "{cleaned_idea}"
# If you ask followup questions, make the questions normal and conversational. Do not ask all the questions at once.'''

# while True:
#     # Get assistant's response
#     response2 = client.chat.completions.create(
#         model="llama-3.2-3b-instruct",
#         # model="mistral-nemo-instruct-2407",
#         messages=[
#             {"role": "system", "content": system_prompt2},
#             *conversation_history,
#             {"role": "user", "content": user_prompt2}
#         ],
#     )
#     duck_response = response2.choices[0].message.content
#     print("\nAssistant:", duck_response)
    
#     # Get user input
#     user_input = input("\nYour response (type 'bye' to end): ")
#     print("--------------------------------")
#     print(f"User input: {user_input}")
    
    
#     if user_input.lower() == 'bye':
#         print("\nGoodbye! Hope we reached a good conclusion for your project!")
#         break
        
#     # Update conversation history
#     conversation_history.append({"role": "assistant", "content": duck_response})
#     conversation_history.append({"role": "user", "content": user_input})
    
#     # Update user prompt for next iteration
#     user_prompt2 = user_input

In [2]:
import whisper
import sounddevice as sd
import numpy as np
import threading
import queue
import time

# Print available audio devices to help with debugging
print("\nAvailable audio devices:")
print(sd.query_devices())

# Initialize Whisper model
model = whisper.load_model("base")

# Audio recording parameters
SAMPLE_RATE = 16000
CHANNELS = 1
CHUNK_DURATION = 3  # seconds
CHUNK_SIZE = int(SAMPLE_RATE * CHUNK_DURATION)

# Create a queue for audio chunks
audio_queue = queue.Queue()

def audio_callback(indata, frames, time, status):
    if status:
        print(f'Status: {status}')
    audio_queue.put(indata.copy())

def process_audio():
    while True:
        try:
            # Get audio chunk from queue
            audio_data = audio_queue.get()
            
            # Convert audio to format expected by Whisper
            audio_data = audio_data.flatten().astype(np.float32)
            
            # Transcribe
            result = model.transcribe(audio_data)
            
            # Print transcription if not empty
            if result["text"].strip():
                print(result["text"])
        except Exception as e:
            print(f"Error in processing: {e}")
            continue

try:
    # Get default input device
    device_info = sd.query_devices(kind='input')
    print(f"\nUsing input device: {device_info['name']}")
    
    # Start recording with explicit device configuration
    stream = sd.InputStream(
        device=None,  # Use default device
        channels=CHANNELS,
        samplerate=SAMPLE_RATE,
        callback=audio_callback,
        blocksize=CHUNK_SIZE,
        dtype=np.float32
    )
    
    # Start processing thread
    processing_thread = threading.Thread(target=process_audio, daemon=True)
    processing_thread.start()
    
    print("\nStarting transcription... Speak into your microphone")
    
    with stream:
        while True:
            time.sleep(0.1)

except KeyboardInterrupt:
    print("\nStopping transcription...")
except Exception as e:
    print(f"\nError: {e}")
    # Print more detailed device information for debugging
    print("\nDetailed device information:")
    for i, device in enumerate(sd.query_devices()):
        print(f"Device {i}: {device}")
finally:
    if 'stream' in locals():
        stream.stop()


Available audio devices:
  0 Mrityunjay’s iPhone Microphone, Core Audio (1 in, 0 out)
> 1 MacBook Pro Microphone, Core Audio (1 in, 0 out)
< 2 MacBook Pro Speakers, Core Audio (0 in, 2 out)


/opt/miniconda3/envs/rag/lib/python3.11/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Using input device: MacBook Pro Microphone

Starting transcription... Speak into your microphone


/opt/miniconda3/envs/rag/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hello, hello.
 Can you hear me?
 whisper is working perfectly fine.
 Thank you.
 What do you mean, he won't check the trans...
 Be nice, prehistoric with our encore and from here we'll be back in Mahayana Yamag名課童 This won't change plans in a few moments Let's see... specify of course that you need to wear shadow of me Okeh, there you go To keep you healthy Take something out of mí Absolutely
 Thank you.
 Nice brief story with this.
 melaто
 புரிக்கிறாய் அம்மா.
 Thank you.
 what the
 Thank you. Thank you.
 Thank you.
 Bye Singapore.
 کے بلroads انroller ایک انہ<|cs|>
 बाहीं का बूल था है?
 We're still thinking I'm going to ask for more.
 ब१व agreeikan

Stopping transcription...


 Só哥 funnel Guaiubá, eu tenho sore, eu tenho sore...


In [26]:
from datetime import datetime
import os
import whisper

class ConversationReport:
    def __init__(self, original_idea):
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.original_idea = original_idea
        self.conversation_exchanges = []
        
    def add_exchange(self, assistant_msg, user_msg):
        self.conversation_exchanges.append({
            "assistant": assistant_msg,
            "user": user_msg
        })
            
    def save_transcript(self, project_name):
        # Create reports directory if it doesn't exist
        os.makedirs("reports", exist_ok=True)
        
        # Generate filename
        filename = f"reports/{project_name}_transcript_{self.timestamp}.txt"
        
        # Save transcript as text file
        with open(filename, 'w') as f:
            f.write(f"Project: {project_name}\n")
            f.write(f"Original Idea: {self.original_idea}\n")
            f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("\nTranscript:\n\n")
            
            for exchange in self.conversation_exchanges:
                f.write(f"Assistant: {exchange['assistant']}\n")
                f.write(f"User: {exchange['user']}\n\n")
            
        return filename

# Modified main code
client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

system_prompt = '''You are a helpful assistant. Your job is to filter the user's text (which is a transcript of a conversation) and remove all filler, unnecessary and unrelated text. You must output text directly, such that it can be fed to another llm model. Fix incorrect transcripts to preserve technical knowledge. Just output '''

transcription = '''I want to build a restaurant management system in Java'''

user_prompt = f'''Here's the transcription, clean it up and preserve technical knowledge and return first person text: "{transcription}"'''

response1 = client.chat.completions.create(
    model="llama-3.2-3b-instruct",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
)

cleaned_idea = response1.choices[0].message.content

# Initialize conversation report
# report = ConversationReport(cleaned_idea)

# Initialize conversation history
conversation_history = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

system_prompt2 = '''You are a helpful assistant whose job is to guide the user in ideating a project or code approach. do not supply the answer but ask questions to guide the user. Keep the conversation going until reaching a concrete implementation plan. do not ask all questions at once, solve iteratively as if its a natural conversation.
'''

user_prompt2 = f'''Here's the user's idea, filtered: "{cleaned_idea}"
If you ask followup questions, make the questions normal and conversational. Do not ask all the questions at once.'''

while True:
    # Get assistant's response
    response2 = client.chat.completions.create(
        model="llama-3.2-3b-instruct",
        messages=[
            {"role": "system", "content": system_prompt2},
            *conversation_history,
            {"role": "user", "content": user_prompt2}
        ],
    )
    duck_response = response2.choices[0].message.content
    print("\nAssistant:", duck_response)
    
    # Get user input
    user_input = input("\nYour response (type 'bye' to end): ")
    print("--------------------------------")
    print(f"User input: {user_input}")
    
    # Add exchange to report
    # report.add_exchange(duck_response, user_input)
    
    if user_input.lower() == 'bye':
        # Save the transcript
        project_name = "restaurant_management"  # You might want to get this from user
        # transcript_file = report.save_transcript(project_name)
        # print(f"\nConversation transcript saved to: {transcript_file}")
        print("\nGoodbye! Hope we reached a good conclusion for your project!")
        break
        
    # Update conversation history
    conversation_history.append({"role": "assistant", "content": duck_response})
    conversation_history.append({"role": "user", "content": user_input})
    
    # Update user prompt for next iteration
    user_prompt2 = user_input


Assistant: That sounds like a great project to work on. So, you'd like to develop a comprehensive restaurant management system using Java. Have you thought about the specific features of the system you'd like to implement? For example, would it handle things like ordering and inventory management, customer database, staff scheduling, or something else?

Assistant: It seems like you've fleshed out some specific features of your restaurant management system. Integrating with DoorDash for delivery and Stripe for payments will definitely enhance the user experience.

Before you start reaching out to restaurant owners, have you considered how your system will handle scalability and security? For example, how will it handle a large number of restaurants or a high volume of transactions? What measures will you take to protect sensitive customer data, such as payment information?

Also, have you thought about the potential costs associated with integrating with DoorDash and Stripe? Will these